# 📋 05. 데이터 취합 & 복사 붙여넣기

> **목표**: 여러 시트 / 여러 파일의 데이터를 **하나로 합치고**, 서식을 보존한 채 복사/붙여넣기 한다.
>
> **왜 중요한가?** — "각 팀별 시트를 하나로 합쳐줘", "이 6개 파일을 종합 시트에 모아줘"  
> 이런 요청이 **실무의 70%**를 차지한다.

---

### 📚 이 챕터에서 배우는 것

| 순서 | 주제 | 실무 적용 |
|:---:|:---|:---|
| 1 | 값만 복사 | 다른 시트로 데이터 옮기기 |
| 2 | 서식 포함 복사 | 원본 서식 그대로 유지 |
| 3 | 수식 복사 | 수식을 그대로 가져오기 |
| 4 | 여러 시트 → 종합 | 부서별 데이터 합산 |
| 5 | 여러 파일 → 하나로 | 월별 보고서 통합 |
| 6 | pandas 연동 | 대용량 분석 → 엑셀 출력 |
| 7 | 🔥 실전 종합 | 교육이수 → 종합표 + 통계 자동 생성 |

---
## 🧠 핵심 이론: xlwings에서 복사하는 3가지 방법

| 방식 | 코드 | 특징 |
|:---|:---|:---|
| **값만 복사** | `.value` 대입 | 가장 빠르고 단순, 서식 X |
| **서식 포함 복사** | `.api.Copy()` + `.api.PasteSpecial()` | VBA 방식, 서식+값 모두 |
| **pandas 경유** | `read → DataFrame → write` | 대용량 분석에 최적 |

### PasteSpecial 상수 정리

| 상수 | 값 | 붙여넣기 대상 |
|:---|:---:|:---|
| xlPasteAll | -4104 | 전체 (값+서식+수식) |
| xlPasteValues | -4163 | 값만 |
| xlPasteFormats | -4122 | 서식만 |
| xlPasteFormulas | -4123 | 수식만 |
| xlPasteColumnWidths | 8 | 열 너비만 |

---
## 0️⃣ 준비: 엑셀 파일 열기

In [ ]:
import xlwings as xw

app = xw.App(add_book=False)
wb = app.books.open('교육이수현황.xlsx')

print(f'✅ 파일 열기 완료: {wb.name}')
print(f'📋 시트 목록: {[s.name for s in wb.sheets]}')

---
## 1️⃣ 값만 복사 (`.value` 대입)

In [ ]:
# 소스와 목적지 시트 지정
src = wb.sheets['마케팅팀']
dst = wb.sheets['영업1팀']

# 단일 셀 복사
dst.range('A1').value = src.range('A1').value

print(f'✅ 단일 셀 복사 완료')
print(f'  원본 (마케팅팀 A1): {src.range("A1").value}')
print(f'  복사된 값 (영업1팀 A1): {dst.range("A1").value}')

In [ ]:
# 범위 복사 (2차원 리스트로 반환)
data = src.range('A2:D6').value

print(f'📊 복사할 데이터 ({len(data)}행):')
for row in data:
    print(f'  {row}')

In [ ]:
# 읽어온 데이터를 다른 시트에 붙여넣기
# dst.range('A2').value = data

print('💡 위 코드의 주석을 해제하면 영업1팀에 데이터가 복사됩니다.')
print('💡 이미 영업1팀에 데이터가 있다면 덮어쓰기 됩니다!')

In [ ]:
# 동적 범위 복사 (데이터가 있는 만큼만 자동으로)
data = src.range('A2').expand('table').value

print(f'📊 동적 범위: {len(data)}행 감지됨')
print(f'  첫 행: {data[0]}')
print(f'  마지막 행: {data[-1]}')

---
## 2️⃣ 수식 복사 vs 값 복사

> ⚠️ `.value`로 읽으면 수식이 아닌 **계산된 값**이 복사됩니다!  
> 수식을 그대로 가져오려면 `.formula`를 사용하세요.

In [ ]:
# 수식이 있는 셀 비교 (C6에 =SUM() 수식이 있다면)
ws = wb.sheets['마케팅팀']

val = ws.range('C6').value       # → 계산된 값 (예: 36.0)
formula = ws.range('C6').formula  # → 수식 문자열 (예: '=SUM(C3:C5)')

print(f'📊 C6 셀:')
print(f'  .value (계산 결과): {val}')
print(f'  .formula (수식 원문): {formula}')

In [ ]:
# 수식을 다른 셀에 복사하기
# dst.range('C6').formula = ws.range('C6').formula

print('💡 수식 복사: dst.range("C6").formula = src.range("C6").formula')
print('💡 .formula는 수식 문자열을 그대로 복사합니다')

---
## 3️⃣ 서식 포함 복사 (API 방식)

> 실제 엑셀의 **Ctrl+C / Ctrl+V**와 동일한 동작

In [ ]:
# 서식까지 통째로 복사 (Copy → PasteSpecial)
# src.range('A2:D6').api.Copy()
# dst.range('A2').api.PasteSpecial(Paste=-4104)  # xlPasteAll = 전체

# 클립보드 정리 (깜빡이는 점선 제거)
# app.api.CutCopyMode = False

print('💡 서식 포함 복사 코드 (주석 해제 후 사용):')
print('  1) src.range("A2:D6").api.Copy()')
print('  2) dst.range("A2").api.PasteSpecial(Paste=-4104)')
print('  3) app.api.CutCopyMode = False  ← 점선 제거')

In [ ]:
# 값과 서식을 따로따로 붙여넣기
# src.range('A2:D6').api.Copy()
# dst.range('A10').api.PasteSpecial(Paste=-4163)   # 값만
# dst.range('A10').api.PasteSpecial(Paste=-4122)   # 서식만
# app.api.CutCopyMode = False

print('💡 PasteSpecial 옵션:')
print('  -4104 = 전체 (값+서식+수식)')
print('  -4163 = 값만')
print('  -4122 = 서식만')
print('  -4123 = 수식만')

---
## 4️⃣ 여러 시트 → 하나로 합치기 (취합)

> 🎯 **실무에서 가장 많이 쓰는 패턴!**  
> 각 팀별 시트 데이터를 '종합' 시트 하나로 합칩니다.

In [ ]:
def merge_sheets_to_one(wb, target_sheet_name='종합'):
    """
    모든 시트의 데이터를 '종합' 시트로 합침
    
    Parameters:
        wb: 워크북 객체
        target_sheet_name: 종합 시트 이름 (기본: '종합')
    """
    
    # 1) 종합 시트 준비 (있으면 초기화, 없으면 생성)
    existing_names = [s.name for s in wb.sheets]
    
    if target_sheet_name in existing_names:
        ws_target = wb.sheets[target_sheet_name]
        ws_target.clear()
        print(f'📋 기존 "{target_sheet_name}" 시트 초기화')
    else:
        ws_target = wb.sheets.add(target_sheet_name, after=wb.sheets[-1])
        print(f'📋 "{target_sheet_name}" 시트 새로 생성')
    
    current_row = 1
    header_written = False
    
    # 2) 각 시트 순회
    for sheet in wb.sheets:
        if sheet.name == target_sheet_name:
            continue  # 종합 시트 자체는 건너뜀
        
        ws = sheet
        print(f'\n📋 {ws.name} 처리 중...')
        
        # 데이터 확인
        if ws.range('A2').value is None and ws.range('B2').value is None:
            print(f'  ⏭️ 빈 시트, 건너뜀')
            continue
        
        # 3) 헤더 복사 (첫 번째 시트에서만)
        if not header_written:
            try:
                header = ws.range('A2').expand('right').value
                if isinstance(header, list):
                    ws_target.range('A1').value = ['팀명'] + header
                else:
                    ws_target.range('A1').value = ['팀명', header]
                
                # 헤더 서식
                ws_target.range('A1').expand('right').font.bold = True
                ws_target.range('A1').expand('right').color = (68, 114, 196)
                ws_target.range('A1').expand('right').font.color = (255, 255, 255)
                
                current_row = 2
                header_written = True
                print(f'  📝 헤더 작성 완료')
            except Exception as e:
                print(f'  ⚠️ 헤더 읽기 실패: {e}')
                continue
        
        # 4) 데이터 읽기 (헤더 제외, 3행부터)
        try:
            data = ws.range('A3').expand('table').value
            
            # 단일 행이면 리스트로 감싸기
            if data is not None and not isinstance(data[0], list):
                data = [data]
            
            if data is None:
                print(f'  ⏭️ 3행부터 데이터 없음')
                continue
            
            # 5) 각 행에 팀명 추가하여 종합 시트에 쓰기
            for row_data in data:
                if isinstance(row_data, list):
                    ws_target.range(f'A{current_row}').value = [ws.name] + row_data
                else:
                    ws_target.range(f'A{current_row}').value = [ws.name, row_data]
                current_row += 1
            
            print(f'  ✅ {len(data)}행 추가')
            
        except Exception as e:
            print(f'  ⚠️ 데이터 읽기 실패: {e}')
    
    # 6) 열 자동 맞춤
    ws_target.range('A1').expand('table').columns.autofit()
    
    total_rows = current_row - 2
    print(f'\n🎉 취합 완료! 총 {total_rows}건 → "{target_sheet_name}" 시트')
    return ws_target


print('✅ merge_sheets_to_one() 함수 정의 완료!')

In [ ]:
# 실행! 모든 시트 데이터를 '종합' 시트로 합치기
ws_total = merge_sheets_to_one(wb, '종합')

print('\n💡 엑셀에서 "종합" 시트를 확인해보세요!')

---
## 5️⃣ 여러 파일 → 하나로 합치기

> 폴더 안에 있는 **모든 `.xlsx` 파일**의 데이터를 하나로 합칩니다.

In [ ]:
# 먼저 현재 폴더의 엑셀 파일 확인
from pathlib import Path

current_folder = Path('.')
excel_files = list(current_folder.glob('*.xlsx'))

print(f'📁 현재 폴더의 엑셀 파일:')
for f in excel_files:
    size_kb = f.stat().st_size / 1024
    print(f'  📄 {f.name} ({size_kb:.1f} KB)')

In [ ]:
def merge_files_to_one(folder_path, output_filename='종합_보고서.xlsx'):
    """
    폴더 내 모든 xlsx 파일의 첫 번째 시트 데이터를 합침
    
    Parameters:
        folder_path: 폴더 경로 (문자열)
        output_filename: 결과 파일명
    """
    app_local = xw.App(add_book=False, visible=False)
    
    # 결과 파일 생성
    wb_out = app_local.books.add()
    ws_out = wb_out.sheets[0]
    ws_out.name = '종합'
    
    folder = Path(folder_path)
    files = sorted(folder.glob('*.xlsx'))
    # 결과 파일 자체는 제외
    files = [f for f in files if f.name != output_filename]
    
    print(f'📁 {folder_path}')
    print(f'📄 {len(files)}개 파일 발견\n')
    
    current_row = 1
    header_written = False
    
    for filepath in files:
        print(f'📄 {filepath.name} 처리 중...')
        
        try:
            wb_src = app_local.books.open(str(filepath.absolute()))
            ws_src = wb_src.sheets[0]  # 첫 번째 시트
            
            # 헤더 (첫 파일에서만)
            if not header_written:
                header = ws_src.range('A1').expand('right').value
                if header is not None:
                    h = header if isinstance(header, list) else [header]
                    ws_out.range('A1').value = ['파일명'] + h
                    ws_out.range('A1').expand('right').font.bold = True
                    current_row = 2
                    header_written = True
            
            # 데이터 (2행부터)
            data_cell = ws_src.range('A2')
            if data_cell.value is not None:
                data = data_cell.expand('table').value
                if not isinstance(data[0], list):
                    data = [data]
                
                for row in data:
                    row_list = row if isinstance(row, list) else [row]
                    ws_out.range(f'A{current_row}').value = [filepath.stem] + row_list
                    current_row += 1
                
                print(f'  ✅ {len(data)}행 추가')
            
            wb_src.close()
            
        except Exception as e:
            print(f'  ❌ 에러: {e}')
    
    # 저장
    output_path = str(folder / output_filename)
    ws_out.range('A1').expand('table').columns.autofit()
    wb_out.save(output_path)
    wb_out.close()
    app_local.quit()
    
    print(f'\n🎉 총 {current_row - 2}건 → {output_path} 저장 완료!')


print('✅ merge_files_to_one() 함수 정의 완료!')
print('💡 사용: merge_files_to_one(r"C:\\내폴더", "종합_보고서.xlsx")')

---
## 6️⃣ pandas와 xlwings 연동

> **pandas = 분석 엔진**, **xlwings = 서식/출력 도구**  
> 둘의 조합이 **실무 최강 콤보**입니다.

In [ ]:
# pandas가 없으면 설치
# pip install pandas

import pandas as pd
print(f'✅ pandas {pd.__version__} 로드 완료')

In [ ]:
# xlwings → pandas (엑셀 데이터를 DataFrame으로 읽기)
ws = wb.sheets['마케팅팀']

# range에서 직접 DataFrame 생성
df = ws.range('A2').expand('table').options(pd.DataFrame, header=1).value

print('📊 DataFrame으로 변환된 데이터:')
print(df)
print(f'\n📍 형태: {df.shape[0]}행 x {df.shape[1]}열')

In [ ]:
# pandas로 간단한 분석
print('📈 기술 통계:')
print(df.describe())

print(f'\n📊 데이터 타입:')
print(df.dtypes)

In [ ]:
# pandas → xlwings (분석 결과를 엑셀에 쓰기)
# 새 시트 생성 후 DataFrame 출력

# 기존 '분석결과' 시트가 있으면 삭제
existing = [s.name for s in wb.sheets]
if '분석결과' in existing:
    wb.sheets['분석결과'].delete()

ws_result = wb.sheets.add('분석결과', after=wb.sheets[-1])

# DataFrame을 통째로 엑셀에 쓰기
ws_result.range('A1').value = '📊 원본 데이터'
ws_result.range('A1').font.bold = True
ws_result.range('A1').font.size = 14
ws_result.range('A2').value = df

# 기술 통계도 같이
ws_result.range('A10').value = '📈 기술 통계'
ws_result.range('A10').font.bold = True
ws_result.range('A10').font.size = 14
ws_result.range('A11').value = df.describe()

ws_result.range('A1').expand('table').columns.autofit()

print('✅ 분석 결과를 "분석결과" 시트에 저장 완료!')
print('💡 엑셀에서 확인해보세요!')

In [ ]:
# 🎯 여러 시트를 하나의 DataFrame으로 합치기 (pandas 방식)

all_data = []
skip_sheets = ['종합', '분석결과']

for sheet in wb.sheets:
    if sheet.name in skip_sheets:
        continue
    
    try:
        temp_df = sheet.range('A2').expand('table').options(pd.DataFrame, header=1).value
        temp_df['팀명'] = sheet.name  # 팀명 열 추가
        all_data.append(temp_df)
        print(f'  ✅ {sheet.name}: {len(temp_df)}행')
    except:
        print(f'  ⏭️ {sheet.name}: 건너뜀')

if all_data:
    combined = pd.concat(all_data, ignore_index=True)
    print(f'\n📊 합쳐진 전체 데이터: {len(combined)}행')
    print(combined)
else:
    print('\n⚠️ 합칠 데이터가 없습니다')

---
## 🔥 7. 실전 종합: 교육이수 → 종합 보고서 + 통계 시트 자동 생성

> 이것이 **실무에서 매일 쓸 수 있는 완성형 코드**입니다.  
> 팀별 데이터를 취합하고, 통계 시트까지 자동으로 만듭니다.

In [ ]:
def create_comprehensive_report(wb):
    """
    팀별 교육이수 데이터를 취합하고,
    종합 보고서 + 통계 시트를 자동 생성
    """
    
    # ==========================
    # 준비: 기존 결과 시트 정리
    # ==========================
    cleanup_sheets = ['종합_최종', '통계']
    for name in cleanup_sheets:
        if name in [s.name for s in wb.sheets]:
            wb.sheets[name].delete()
    
    # ==========================
    # STEP 1: 데이터 취합
    # ==========================
    print('📊 STEP 1: 데이터 취합')
    print('=' * 40)
    
    ws_total = wb.sheets.add('종합_최종', after=wb.sheets[-1])
    
    # 헤더
    ws_total.range('A1').value = ['팀명', '성명', '교육시간', '코드']
    ws_total.range('A1:D1').font.bold = True
    ws_total.range('A1:D1').color = (68, 114, 196)
    ws_total.range('A1:D1').font.color = (255, 255, 255)
    ws_total.range('A1:D1').api.HorizontalAlignment = -4108
    
    row = 2
    team_stats = {}  # 팀별 통계 수집
    skip_list = ['종합', '종합_최종', '통계', '분석결과']
    
    for sheet in wb.sheets:
        if sheet.name in skip_list:
            continue
        
        try:
            if sheet.range('A2').value is None and sheet.range('B2').value is None:
                continue
            
            data = sheet.range('A2').expand('table').value
            if data is None:
                continue
            if not isinstance(data[0], list):
                data = [data]
            
            team_hours = []
            member_count = 0
            
            for d in data:
                # 헤더 행 건너뛰기 (숫자가 아닌 행)
                values = list(d)
                
                # 교육시간 찾기 (숫자인 열)
                hours = None
                for v in values:
                    if isinstance(v, (int, float)) and v > 0:
                        hours = v
                        break
                
                if hours is not None:
                    ws_total.range(f'A{row}').value = sheet.name
                    ws_total.range(f'B{row}').value = values[1] if len(values) > 1 else values[0]
                    ws_total.range(f'C{row}').value = hours
                    if len(values) > 2:
                        ws_total.range(f'D{row}').value = values[-1]
                    
                    team_hours.append(hours)
                    member_count += 1
                    row += 1
            
            if team_hours:
                team_stats[sheet.name] = {
                    'count': member_count,
                    'total': sum(team_hours),
                    'avg': sum(team_hours) / len(team_hours)
                }
                print(f'  ✅ {sheet.name}: {member_count}명, 합계 {sum(team_hours)}시간')
            
        except Exception as e:
            print(f'  ⚠️ {sheet.name}: {e}')
    
    # 종합 시트 마무리
    ws_total.range('A1').expand('table').columns.autofit()
    ws_total.range('C2:C100').number_format = '#,##0'
    
    # 테두리
    last_row = row - 1
    if last_row >= 2:
        rng = ws_total.range(f'A1:D{last_row}').api
        for bid in range(7, 13):
            try:
                rng.Borders(bid).LineStyle = 1
                rng.Borders(bid).Weight = 2
            except:
                pass
    
    # ==========================
    # STEP 2: 통계 시트 생성
    # ==========================
    print(f'\n📈 STEP 2: 통계 시트 생성')
    print('=' * 40)
    
    ws_stats = wb.sheets.add('통계', after=ws_total)
    
    # 제목
    ws_stats.range('A1:D1').merge()
    ws_stats.range('A1').value = '📊 팀별 교육이수 통계'
    ws_stats.range('A1').font.size = 16
    ws_stats.range('A1').font.bold = True
    ws_stats.range('A1').font.name = '맑은 고딕'
    ws_stats.range('A1').color = (47, 85, 151)
    ws_stats.range('A1').font.color = (255, 255, 255)
    ws_stats.range('A1').row_height = 45
    ws_stats.range('A1:D1').api.HorizontalAlignment = -4108
    
    # 헤더
    ws_stats.range('A2').value = ['팀명', '인원수', '총 교육시간', '평균 교육시간']
    ws_stats.range('A2:D2').font.bold = True
    ws_stats.range('A2:D2').color = (68, 114, 196)
    ws_stats.range('A2:D2').font.color = (255, 255, 255)
    ws_stats.range('A2:D2').api.HorizontalAlignment = -4108
    ws_stats.range('A2').row_height = 30
    
    # 데이터
    stat_row = 3
    for team, stats in team_stats.items():
        ws_stats.range(f'A{stat_row}').value = [
            team,
            stats['count'],
            stats['total'],
            round(stats['avg'], 1)
        ]
        print(f'  📊 {team}: {stats["count"]}명, 합계 {stats["total"]}h, 평균 {stats["avg"]:.1f}h')
        stat_row += 1
    
    # 합계 행
    ws_stats.range(f'A{stat_row}').value = '합계'
    ws_stats.range(f'A{stat_row}').font.bold = True
    ws_stats.range(f'B{stat_row}').value = f'=SUM(B3:B{stat_row-1})'
    ws_stats.range(f'C{stat_row}').value = f'=SUM(C3:C{stat_row-1})'
    ws_stats.range(f'D{stat_row}').value = f'=AVERAGE(D3:D{stat_row-1})'
    ws_stats.range(f'A{stat_row}:D{stat_row}').color = (255, 242, 204)
    ws_stats.range(f'A{stat_row}:D{stat_row}').font.bold = True
    
    # 숫자 서식
    ws_stats.range(f'C3:C{stat_row}').number_format = '#,##0'
    ws_stats.range(f'D3:D{stat_row}').number_format = '0.0'
    
    # 테두리
    rng = ws_stats.range(f'A2:D{stat_row}').api
    for bid in range(7, 13):
        try:
            rng.Borders(bid).LineStyle = 1
            rng.Borders(bid).Weight = 2
        except:
            pass
    
    # 열 자동 맞춤
    ws_stats.range(f'A1:D{stat_row}').columns.autofit()
    
    print(f'\n{"="*40}')
    print(f'🎉 종합 보고서 생성 완료!')
    print(f'  📋 종합_최종 시트: {row - 2}건 데이터')
    print(f'  📊 통계 시트: {len(team_stats)}개 팀 통계')
    print(f'{"="*40}')


print('✅ create_comprehensive_report() 함수 정의 완료!')

In [ ]:
# 🚀 실행! 종합 보고서 + 통계 자동 생성
create_comprehensive_report(wb)

# 저장
wb.save()
print('\n💾 파일 저장 완료!')
print('💡 엑셀에서 "종합_최종"과 "통계" 시트를 확인해보세요!')

---
## 💾 마무리

In [ ]:
# 엑셀 닫기 (확인 후 실행)
# app.quit()

print('💡 모든 작업 확인 후 app.quit() 실행하세요')

---
## ⚠️ 흔한 에러와 해결

| 증상 | 원인 | 해결 |
|:---|:---|:---|
| `data`가 `None` | 빈 시트에서 expand() 시도 | expand 전에 None 체크 |
| 1행짜리 데이터가 리스트가 아닌 단일값 | 한 행이면 1차원 반환 | `if not isinstance(data[0], list): data = [data]` |
| PasteSpecial이 안 먹힘 | Copy 이후 다른 작업이 끼어듦 | Copy → PasteSpecial 사이에 다른 작업 X |
| "시트 이름 중복" 에러 | 이미 같은 이름의 시트 존재 | 생성 전에 존재 여부 확인 후 삭제 |
| pandas import 에러 | pandas 미설치 | `pip install pandas` |

---
## 📝 연습 문제

1. 종합 시트의 데이터를 **교육시간 높은 순으로 정렬**해보세요 (pandas 사용)
2. `merge_files_to_one()` 함수에 **중복 행 제거** 로직을 추가해보세요
3. pandas를 사용해서 **팀별 교육시간 평균 막대 차트**를 만들어보세요 (matplotlib)

In [ ]:
# 🏋️ 연습 1: pandas로 교육시간 높은 순 정렬
# 아래 코드를 완성해보세요!

# ws_total = wb.sheets['종합_최종']
# df = ws_total.range('A1').expand('table').options(pd.DataFrame, header=1).value
# df_sorted = df.sort_values(by='???', ascending=???)
# print(df_sorted)

---
## 📌 다음 챕터 예고

> **06. 실전 자동화 프로젝트**  
> PDF 변환, 차트 생성, 이메일 발송, 스케줄링까지 — 완전한 업무 자동화 파이프라인